#기본 준비

## 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 디렉토리 생성

In [33]:
!mkdir datafile

In [ ]:
!mkdir templates

In [ ]:
!mkdir static

In [ ]:
!mkdir sounds

In [ ]:
!mkdir lib

In [ ]:
!mkdir img

준비된 파일들은 생성된 폴더에 직접 추가


#Naver Clova API

###모듈 임포트

In [2]:
import hashlib
import hmac
import base64
import time
import requests
import json
import os
import sys
import urllib.request
import shutil

## Clova 챗봇

### 챗봇 클래스 정의

In [3]:
class ChatbotMessageSender:

    # chatbot api gateway url
    ep_path = 'https://3svep1xfda.apigw.ntruss.com/custom/v1/6887/558fe5674f162b84dd4bd83029e086ea1d96f9457826544f95b49924aa453fba'
    # chatbot custom secret key
    secret_key = 'VGJiQ2NTUkF3SWJUeUZ3QXlYdW9nZ0dUTVdOY2Vuc1c='

    def req_message_send(self, text):

        timestamp = self.get_timestamp()
        request_body = {
            'version': 'v2',
            'userId': 'U47b00b58c90f8e47428af8b7bddcda3d1111111',
            'timestamp': timestamp,
            'bubbles': [
                {
                    'type': 'text',
                    'data': {
                        'description': text
                    }
                }
            ],
            'event': 'send'
        }

        ## Request body
        encode_request_body = json.dumps(request_body).encode('UTF-8')

        ## make signature
        signature = self.make_signature(self.secret_key, encode_request_body)

        ## headers
        custom_headers = {
            'Content-Type': 'application/json;UTF-8',
            'X-NCP-CHATBOT_SIGNATURE': signature
        }

        # print("## Timestamp : ", timestamp)
        # print("## Signature : ", signature)
        # print("## headers ", custom_headers)
        # print("## Request Body : ", encode_request_body)

        ## POST Request
        response = requests.post(headers=custom_headers, url=self.ep_path, data=encode_request_body)

        return response

    @staticmethod
    def get_timestamp():
        timestamp = int(time.time() * 1000)
        return timestamp

    @staticmethod
    def make_signature(secret_key, request_body):

        secret_key_bytes = bytes(secret_key, 'UTF-8')

        signing_key = base64.b64encode(hmac.new(secret_key_bytes, request_body, digestmod=hashlib.sha256).digest())

        return signing_key

In [4]:
#챗봇 함수
def APIchat(text):
  if __name__ == '__main__':

      res = ChatbotMessageSender().req_message_send(text)
      if(res.status_code == 200):
          
          return(res.json()['bubbles'][0]['data']['description'])
          # print(res.read().decode("UTF-8"))

## TTS(음성 답변용)

In [5]:
def answer_voice(text):
  client_id = "zyqwh8x83v"
  client_secret = "gE21EZDEXp8esM8K2JYUMFXllkanun0Jd6UA7OXG"
  encText = urllib.parse.quote(text)
  data = "speaker=nara&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
  url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
  request = urllib.request.Request(url)
  request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
  request.add_header("X-NCP-APIGW-API-KEY",client_secret)
  response = urllib.request.urlopen(request, data=data.encode('utf-8'))
  rescode = response.getcode()
  if(rescode==200):
      response_body = response.read()
      with open('answer.mp3', 'wb') as f:
          f.write(response_body)
      source = 'answer.mp3'
      #출력될 음성은 로컬에서 저장된 후 자바스크립트를 통해 재생됨
      destination = '/content/sounds/answer.mp3'
      shutil.move(source, destination)
  else:
    errormsg = ("Error Code:" + rescode)
    return errormsg

## STT(음성 입력용)

In [6]:
# 현 코드에서는 미사용
def stt():
  import sys
  import requests
  client_id = "zyqwh8x83v"
  client_secret = "gE21EZDEXp8esM8K2JYUMFXllkanun0Jd6UA7OXG"
  lang = "Kor" # 언어 코드 ( Kor, Jpn, Eng, Chn )
  url = "https://naveropenapi.apigw.ntruss.com/recog/v1/stt?lang=" + lang
  data = open('/content/sounds/answer.mp3', 'rb')
  headers = {
      "X-NCP-APIGW-API-KEY-ID": client_id,
      "X-NCP-APIGW-API-KEY": client_secret,
      "Content-Type": "application/octet-stream"
  }
  response = requests.post(url,  data=data, headers=headers)
  rescode = response.status_code
  if(rescode == 200):
      restext = (eval(response.text))['text']
      return restext
  else:
      return("Error : " + response.text)

In [7]:
#네이버 클라우드플랫폼 챗봇 함수(str 형태로 메시지를 입력하면 동작)
APIchat("농심을 대표하는 라면의 이름?")

'100여 개국'

# SBERT Transformer 이용한 챗봇 제작

## 사전 준비

### 트랜스포머 인스톨

In [8]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.3 MB/s 
     |████████████████████████████████| 4.0 MB 38.6 MB/s 
     |████████████████████████████████| 1.2 MB 31.7 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
     |████████████████████████████████| 6.6 MB 36.7 MB/s 
     |████████████████████████████████| 596 kB 41.7 MB/s 
     |████████████████████████████████| 895 kB 43.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=7c1b3ee840a1bbfb7da88ecba449013eaf061688b80059393cff13056539ba82
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### 모듈 임포트 및 SBERT 모델 불러오기

In [9]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer
import pickle

In [10]:
#센텐스트랜스포머의 다국어 모델 로드
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

##챗봇 구성

### 일상회화형 챗봇

#### 챗봇 구성하기

In [11]:
# 일상대화형 대화셋 로드
# urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
# train_data = pd.read_csv('ChatBotData.csv')
# train_data.head()

In [12]:
# 로드된 데이터셋에 모델 기반으로 임베딩 벡터 컬럼 추가
# train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

####함수 정의

In [11]:
# 코사인 유사도 비교함수 정의
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [12]:
# 입력된 질문의 코사인 유사도를 구하여 데이터프레임에서 답변을 출력하는 함수 정의
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

#### 만들어져 있는 챗봇 데이터 불러오기

In [13]:
# 일상회화형 챗봇용 데이터프레임 피클 로드(해당 피클은 트랜스포머의 버전에 따라 정확도가 달라질 수 있음)
with open("/content/datafile/SBERTchatbotNorm.pkl","rb") as fr:
    train_data = pickle.load(fr)

### korquard 기반 질의응답형 챗봇

#### 챗봇 구성하기

In [17]:
# 전처리된 Q&A 데이터 불러오기
# train_data2 = pd.read_csv('/content/datafile/KorQuAD_2.1_data_all_qna.csv', encoding='utf-8')

In [18]:
# 로드된 데이터셋에 모델 기반으로 임베딩 벡터 컬럼 추가
# train_data2['embedding'] = train_data2.apply(lambda row: model.encode(row.Q), axis = 1)

#### 만들어져 있는 챗봇 데이터 불러오기

In [14]:
# 질의응답형 챗봇용 데이터프레임 피클 로드
with open("/content/datafile/SBERTchatbotWiki.pkl","rb") as fr:
    train_data2 = pickle.load(fr)

In [15]:
# 질의응답형 챗봇용 출력함수 정의
def return_answer2(question):
    embedding = model.encode(question)
    train_data2['score'] = train_data2.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data2.loc[train_data2['score'].idxmax()]['A']

#keyBERT와 SBERT를 사용한 챗봇 제작

In [16]:
# 필요한 모듈, 데이터

!pip install sentence_transformers
!pip install konlpy

import pandas as pd
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm


# 데이터
df = pd.read_pickle('/content/datafile/split_by_doc.pkl')
#사전 준비된 데이터 중 1000, 100000, all 중에서 업로드 된 파일 선택
df2 = pd.read_pickle('/content/datafile/SBERT_final_1000.pkl')
with open('/content/datafile/sbert_stop_words.txt', 'r') as file:
    string = file.read()
    stop_words = string.split('\n')

# 모델, okt 저장
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
okt = Okt()

     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 448 kB 44.9 MB/s 


In [17]:
# 질문 -> 제목, 소제목
# output : str, li (제목, 소제목 리스트)

def find_titles(data, stop_words, question):

  # 질문 -> 제목
  tokenized_doc = okt.pos(question) # 질문 토큰화
  tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if ((word[1] == 'Number') | (word[1] == 'Noun')) & (word[0] not in stop_words)])
  
  # 질문 토큰 embedding
  n_gram_range = (2, 3)
  try:
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([tokenized_nouns]) 
    qas_candidates = count.get_feature_names_out()
    qas_embedding = model.encode(qas_candidates)
  except: 
    qas_candidates = np.array([tokenized_nouns], dtype = object)
    qas_embedding = model.encode(qas_candidates)
    
  # 질문 embedding, 단어들과 코사인 유사도 확인
  qas_doc_embedding = model.encode([question])
  qas_candidate_embeddings = model.encode(qas_candidates)

  top_n = 1
  distances = cosine_similarity(qas_doc_embedding, qas_candidate_embeddings)
  keywords = [qas_candidates[index] for index in distances.argsort()[0][-top_n:]]
  keyword = keywords[0] # top_n = 1인 경우

  # 글자 수 일치하는 문서 추출
  key_len = len(keyword.replace(' ',''))


  # 제목 리스트에서 코사인 유사도 확인
  distances_li = []
  keywords_li = []
  title_li = list(data['title'])


  idx_li = []
  j = 0
  for i in tqdm(range(len(title_li))): # 함수 2번째 버전
    if data.loc[i,'title_len'] == key_len:
      tit = title_li[i]
      idx_li.append(i)
      distances = cosine_similarity(qas_embedding, data.loc[i,'embedding'])
      distances_li.append(distances)
      keywords_li.append(max(map(max, distances_li[j])))
      max_id = keywords_li.index(max(keywords_li))
      title_idx = idx_li[max_id]
      j+=1

  title = title_li[title_idx] # 코사인 유사도 가장 높은 단어를 제목으로 선정


  # 제목 -> 소제목
  dt = data[data['title'] == title]
  subs = dt['sub_name'].values[0]
  subs_idx = dt['sub_idx'].values[0]

  li = ['기본']
  for i in range(len(subs)-1): # 내용 있는 소제목만 출력
    if subs_idx[i+1] - subs_idx[i] != 1:
      li.append(subs[i])
  if len(subs) > 0:
    li.append(subs[len(subs)-1])

  return title, li

In [18]:
find_titles(df2, stop_words, "지미 카터에 대해 알려줘")

  0%|          | 0/100000 [00:00<?, ?it/s]

('지미 카터', ['기본', '어린 시절', '정계 입문', '대통령 재임', '외교 정책', '퇴임 이후', '평가'])

In [19]:
# 제목, 소제목 -> 내용
# output : str (소제목에 해당하는 내용)

def find_content(data, split_data, title, subtitle):
  dt = data[data['title'] == title]

  # 1) desc -> desc 출력
  if subtitle == '기본':
    return dt['desc'].values[0]

  # 2) 그 외 특정 목차
  # 해당 목차의 내용을 출력
  else:
    try:
      j = dt['sub_name'].values[0].index(subtitle)
      st = dt['sub_idx'].values[0][j] + 1
      try:
        ed = dt['sub_idx'].values[0][j+1]
      except:
        ed = -1
      output = ' '.join(split_data[title].split('\n')[st:ed])
      return output
    except:
      return '잘못된 입력입니다.'

In [20]:
find_content(df2, df, "지미 카터", "기본")

'제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.'

#API 웹서비스 구현

### 사전 준비

#### FastAPI 인스톨

In [21]:
!pip install fastapi nest-asyncio pyngrok uvicorn aiofiles python-multipart

     |████████████████████████████████| 54 kB 1.8 MB/s 
     |████████████████████████████████| 745 kB 10.6 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 10.9 MB 44.0 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=ab7fd94f355755c3376bf2ac105ce1ecd52392428bb7cc3aefe8e918a3732e1f
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=e957503a9f23f4f4db4f4be2b0d2e72378b524f3edc37ffbde1f8a9d87ac3e59
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built pyngrok python-multipart


#### 모듈 임포트

In [22]:
from fastapi import FastAPI, Request, Form, File, UploadFile
from fastapi.responses import HTMLResponse
from fastapi.responses import PlainTextResponse
from fastapi.responses import JSONResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import nest_asyncio
from pyngrok import ngrok

import uvicorn

###외부API request 모듈

In [28]:
!pip install requests

In [29]:
import requests

### FastAPI 구성

T5모델은 별도의 노트북에서 실행 후 url을 받아올 것

In [30]:
from starlette.responses import Response
# FastAPI
from fastapi.staticfiles import StaticFiles

T5url = "T5 모델의 웹서비스 url"

app=FastAPI()
templates = Jinja2Templates(directory="templates")
app.mount('/static', StaticFiles(directory='static'), name='static')
app.mount('/sounds', StaticFiles(directory='sounds'), name='sounds')
app.mount('/templates', StaticFiles(directory='templates'), name='templates')
app.mount('/lib', StaticFiles(directory='lib'), name='lib')
app.mount('/img', StaticFiles(directory='img'), name='img')

@app.get('/', response_class=HTMLResponse)
async def read_topic(request: Request):
  return templates.TemplateResponse("Mainpage.html", {"request": request})

@app.get('/mainpage', response_class=HTMLResponse)
async def read_topic(request: Request):
  return templates.TemplateResponse("Mainpage.html", {"request": request})

@app.get('/test', response_class=HTMLResponse)
async def read_topic(request: Request):
  return templates.TemplateResponse("test.html", {"request": request})

@app.get('/T5chatbot', response_class=HTMLResponse)
async def get_topic(request: Request):
  return templates.TemplateResponse("T5chatbot.html", {"request": request})

@app.get('/chatbot', response_class=HTMLResponse)
async def get_topic(request: Request):
  return templates.TemplateResponse("voicechat.html", {"request": request})

@app.get('/talkchatbot', response_class=HTMLResponse)
async def get_topic(request: Request):
  return templates.TemplateResponse("talkchat.html", {"request": request})

@app.get('/NVchatbot', response_class=HTMLResponse)
async def get_topic(request: Request):
  return templates.TemplateResponse("NVchatbot.html", {"request": request})

@app.get('/SBERTchatbot', response_class=HTMLResponse)
async def get_topic(request: Request):
  return templates.TemplateResponse("SbertChatbot.html", {"request": request})

@app.post('/chatresult', response_class=PlainTextResponse)
async def get_reply(request: Request, message2: str = Form(...)):
  reply = return_answer2(message2)
  answer_voice(reply)
  return reply

@app.post('/NVchatresult', response_class=PlainTextResponse)
async def get_reply(request: Request, message2: str = Form(...)):
  reply = APIchat(message2)
  answer_voice(reply)
  return reply

@app.post('/talkchatresult', response_class=PlainTextResponse)
async def get_reply(request: Request, message2: str = Form(...)):
  reply = return_answer(message2)
  answer_voice(reply)
  return reply

@app.post('/NVchatresult', response_class=PlainTextResponse)
async def get_reply(request: Request, message2: str = Form(...)):
  reply = APIchat(message2)
  answer_voice(reply)
  return reply

@app.post('/T5chatresult', response_class=PlainTextResponse)
async def get_reply(request: Request, message2: str = Form(...)):
  response = requests.post(T5url+"/T5api",
                           data={"message2":message2})
  reply = response.text
  answer_voice(reply)
  return reply

@app.post('/sberttitle', response_class=JSONResponse)
async def get_reply(request: Request, message2: str = Form(...)):
  title = find_titles(df2, stop_words, message2)[0]
  subtitle = str(find_titles(df2, stop_words, message2)[1]).replace("'","").strip("[").strip("]")
  pmes = "찾으신 "+ title +"에 대한 정보입니다."
  answer_voice(pmes)
  return {"pmes":pmes, "subtitle":subtitle, "title":title}

@app.post('/sbertresult', response_class=PlainTextResponse)
async def get_reply(request: Request, titlesubmit: str = Form(...), subtitle: str = Form(...)):
  reply = find_content(df2, df, titlesubmit, subtitle)
  answer_voice(reply)
  return reply
# @app.post("/chatresult", response_class=JSONResponse)
# async def get_reply(request: Request, message: str = Form(...)):
#   reply = return_answer(message)
#   return [{"reply": reply}]

In [31]:
# ngrok 토큰 (가능하면 본인 계정을 이용할 것, ngroktoken 위치에 토큰 입력 후 셀 실행)
!ngrok authtoken ngroktoken

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


###실행

In [32]:
ngrok_tunnel = ngrok.connect(8000)
print ('Public URL:', ngrok_tunnel.public_url) 
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

Public URL: http://84aa-35-188-68-150.ngrok.io


INFO:     Started server process [58]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     119.207.144.241:0 - "GET / HTTP/1.1" 200 OK
INFO:     1.254.179.100:0 - "GET / HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET / HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /bonobono.png HTTP/1.1" 200 OK
INFO:     1.254.179.100:0 - "GET /bonobono.png HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /T5chatbot HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /style.css HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /rivescript.js HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /jquery.min.js HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /code.js HTTP/1.1" 200 OK
INFO:     182.172.56.199:0 - "GET /T5chatbot HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /pop.mp3 HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /chatbotRiveScript.html HTTP/1.1" 200 OK
INFO:     182.172.56.199:0 - "GET /jquery.min.js HTTP/1.1" 200 OK
INFO:     182.172.56.199:0 - "GET /code.js HTTP/1.1" 200 OK

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

...

INFO:     39.127.112.106:0 - "GET /answer.mp3 HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /SBERTchatbot HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /sbertcode.js HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /favicon.ico HTTP/1.1" 304 Not Modified
INFO:     119.207.144.241:0 - "GET /chatbotRiveScript.html HTTP/1.1" 304 Not Modified
INFO:     119.207.144.241:0 - "GET /pop.mp3 HTTP/1.1" 304 Not Modified
INFO:     119.207.144.241:0 - "GET / HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET / HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /bonobono.png HTTP/1.1" 304 Not Modified
INFO:     119.207.144.241:0 - "GET /SBERTchatbot HTTP/1.1" 200 OK


  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

INFO:     119.207.144.241:0 - "POST /sberttitle HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /answer.mp3 HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "POST /sbertresult HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /answer.mp3 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [58]
